<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/Fast_Random_Fade_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Combined Script for UV Exposure Simulation and Color Fading Prediction

This script integrates UV exposure simulation with a machine learning model to predict
color fading in images. It allows users to upload a LAB color dataset and an image,
simulate UV and visible light exposure, predict color fading, and analyze the results
through various visualizations.

Author: [Your Name]
Date: YYYY-MM-DD
"""

# -----------------------------
# Step 0: Install and Import Necessary Libraries
# -----------------------------

import sys
import subprocess
import importlib
import logging  # Importing logging before using it

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Function to install packages
def install_packages(packages):
    for package in packages:
        if package == 'google.colab':
            continue  # Skip installation as it's already available in Colab
        try:
            importlib.import_module(package)
            logging.info(f"Package '{package}' is already installed.")
        except ImportError:
            print(f"Installing package: {package}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages (excluding 'google.colab')
required_packages = [
    'colormath',
    'scikit-image',
    'numpy',
    'pandas',
    'matplotlib',
    'Pillow',
    'scipy',
    'sklearn'
]

install_packages(required_packages)

# Import necessary libraries after ensuring they are installed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from scipy.spatial import KDTree
from google.colab import files
from skimage import color
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# -----------------------------
# Step 1: Data Upload and Preprocessing
# -----------------------------

def upload_file(prompt_message, file_types=None):
    """
    Prompt the user to upload a file and return the filename.

    :param prompt_message: Message to display to the user.
    :param file_types: Tuple of allowed file extensions.
    :return: Filename of the uploaded file.
    """
    print(prompt_message)
    uploaded = files.upload()
    if not uploaded:
        logging.error("No file uploaded.")
        sys.exit(1)
    filename = next(iter(uploaded))
    if file_types:
        if not filename.lower().endswith(file_types):
            logging.error(f"Uploaded file must be one of the following types: {file_types}")
            sys.exit(1)
    logging.info(f"Uploaded file: {filename}")
    return filename

def load_and_clean_dataset(csv_filename):
    """
    Load the LAB color dataset from a CSV file and clean it.

    :param csv_filename: Name of the CSV file.
    :return: Cleaned pandas DataFrame.
    """
    try:
        dataset = pd.read_csv(csv_filename)
        required_columns = {'L', 'A', 'B', 'Color Name'}
        if not required_columns.issubset(dataset.columns):
            missing = required_columns - set(dataset.columns)
            logging.error(f"Dataset is missing required columns: {missing}")
            sys.exit(1)
        dataset = dataset.replace([np.inf, -np.inf], np.nan).dropna(subset=['L', 'A', 'B'])
        logging.info(f"Dataset loaded with {len(dataset)} entries after cleaning.")
        return dataset
    except Exception as e:
        logging.error(f"Failed to load dataset: {e}")
        sys.exit(1)

def upload_and_process_image():
    """
    Upload an image file and convert it to LAB color space.

    :return: Original PIL Image and LAB numpy array.
    """
    image_filename = upload_file("Please upload the image file you want to analyze.", file_types=('.png', '.jpg', '.jpeg'))
    try:
        image = Image.open(image_filename).convert('RGB')
        image_array = np.array(image).astype(np.float32) / 255.0
        lab_image = color.rgb2lab(image_array)
        logging.info(f"Image '{image_filename}' loaded and converted to LAB color space.")
        return image, lab_image
    except Exception as e:
        logging.error(f"Failed to process image: {e}")
        sys.exit(1)

# -----------------------------
# Step 2: Machine Learning Model Training
# -----------------------------

def create_synthetic_data(num_samples=1000, random_seed=42):
    """
    Generate synthetic data simulating color fading under various lux hours.

    :param num_samples: Number of synthetic samples to generate.
    :param random_seed: Seed for reproducibility.
    :return: pandas DataFrame with synthetic data.
    """
    np.random.seed(random_seed)
    L_fading = np.random.normal(loc=0, scale=5, size=num_samples)
    A_fading = np.random.normal(loc=0, scale=2, size=num_samples)
    B_fading = np.random.normal(loc=0, scale=2, size=num_samples)
    lux_hours = np.random.uniform(low=1000, high=100000, size=num_samples)  # Simulated lux hours
    data = pd.DataFrame({
        'lux_hours': lux_hours,
        'L_fading': L_fading,
        'A_fading': A_fading,
        'B_fading': B_fading
    })
    logging.info(f"Synthetic data generated with {num_samples} samples.")
    return data

def train_ml_model(synthetic_data):
    """
    Train a MultiOutput Random Forest Regressor to predict color fading.

    :param synthetic_data: pandas DataFrame with synthetic data.
    :return: Trained model, scaler, and evaluation metrics.
    """
    X = synthetic_data[['lux_hours']]
    Y = synthetic_data[['L_fading', 'A_fading', 'B_fading']]

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    rf_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
    rf_model.fit(X_train_scaled, Y_train)
    logging.info("Random Forest model trained.")

    Y_pred = rf_model.predict(X_test_scaled)
    mse = mean_squared_error(Y_test, Y_pred)
    logging.info(f"Mean Squared Error for Fading Prediction: {mse:.4f}")

    return rf_model, scaler, mse

# -----------------------------
# Step 3: Exposure Simulation Functions
# -----------------------------

def simulate_uv_exposure(lab_image, exposure_years=1, exposure_factor=36.5, max_exposure_hours=144):
    """
    Simulate UV exposure by adjusting LAB color values.

    :param lab_image: LAB color numpy array of the image.
    :param exposure_years: Number of exposure years to simulate.
    :param exposure_factor: Exposure factor per year.
    :param max_exposure_hours: Maximum exposure hours for full effect.
    :return: UV-exposed LAB image.
    """
    lab_exposed = lab_image.copy()
    current_exposure_hours = exposure_years * exposure_factor
    exposure_ratio = min(current_exposure_hours / max_exposure_hours, 1.0)

    # Increase yellowing effect on the b* axis by 20%
    lab_exposed[:, :, 2] += exposure_ratio * (lab_exposed[:, :, 2] * 0.2)
    lab_exposed[:, :, 2] = np.clip(lab_exposed[:, :, 2], -128, 127)

    logging.info(f"Simulated UV exposure for {exposure_years} years (Exposure Ratio: {exposure_ratio:.2f}).")
    return lab_exposed

def simulate_visible_light_exposure(lab_image, exposure_years=1, exposure_factor=36.5, max_exposure_hours=144):
    """
    Simulate visible light exposure by adjusting LAB color values.

    :param lab_image: LAB color numpy array of the image.
    :param exposure_years: Number of exposure years to simulate.
    :param exposure_factor: Exposure factor per year.
    :param max_exposure_hours: Maximum exposure hours for full effect.
    :return: Visible light-exposed LAB image.
    """
    lab_exposed = lab_image.copy()
    current_exposure_hours = exposure_years * exposure_factor
    exposure_ratio = min(current_exposure_hours / max_exposure_hours, 1.0)

    # Decrease lightness (L*) by 5%
    lab_exposed[:, :, 0] -= exposure_ratio * (lab_exposed[:, :, 0] * 0.05)

    # Subtle shift towards neutral for a* and b* axes by reducing them by 3%
    lab_exposed[:, :, 1] -= exposure_ratio * (lab_exposed[:, :, 1] * 0.03)
    lab_exposed[:, :, 2] -= exposure_ratio * (lab_exposed[:, :, 2] * 0.03)

    # Clip LAB values to valid ranges
    lab_exposed[:, :, 0] = np.clip(lab_exposed[:, :, 0], 0, 100)
    lab_exposed[:, :, 1] = np.clip(lab_exposed[:, :, 1], -128, 127)
    lab_exposed[:, :, 2] = np.clip(lab_exposed[:, :, 2], -128, 127)

    logging.info(f"Simulated Visible Light exposure for {exposure_years} years (Exposure Ratio: {exposure_ratio:.2f}).")
    return lab_exposed

# -----------------------------
# Step 4: Prediction and Application of Fading
# -----------------------------

def predict_fading(model, scaler, lux_hours):
    """
    Predict color fading based on lux hours using the trained model.

    :param model: Trained ML model.
    :param scaler: Scaler used for feature scaling.
    :param lux_hours: Array-like lux hours values.
    :return: Predicted fading values.
    """
    try:
        lux_scaled = scaler.transform(lux_hours)
        predicted_fading = model.predict(lux_scaled)
        logging.info(f"Predicted Fading for {lux_hours.flatten()[0]} lux hours: L_fading={predicted_fading[0][0]:.2f}, "
                     f"A_fading={predicted_fading[0][1]:.2f}, B_fading={predicted_fading[0][2]:.2f}")
        return predicted_fading
    except Exception as e:
        logging.error(f"Failed to predict fading: {e}")
        sys.exit(1)

def apply_fading(lab_image, fading_values):
    """
    Apply predicted fading to the LAB image.

    :param lab_image: Original LAB color numpy array.
    :param fading_values: Array-like fading values for L, A, B.
    :return: Faded LAB image.
    """
    lab_faded = lab_image.copy()
    lab_faded[:, :, 0] += fading_values[0]
    lab_faded[:, :, 1] += fading_values[1]
    lab_faded[:, :, 2] += fading_values[2]

    # Clip LAB values to valid ranges
    lab_faded[:, :, 0] = np.clip(lab_faded[:, :, 0], 0, 100)
    lab_faded[:, :, 1] = np.clip(lab_faded[:, :, 1], -128, 127)
    lab_faded[:, :, 2] = np.clip(lab_faded[:, :, 2], -128, 127)

    logging.info("Applied predicted fading to LAB image.")
    return lab_faded

# -----------------------------
# Step 5: Visualization and Analysis
# -----------------------------

def compute_delta_e(lab1, lab2):
    """
    Compute the Delta-E (∆E) between two LAB colors using the CIE76 formula.

    :param lab1: Array-like LAB values [L, A, B].
    :param lab2: Array-like LAB values [L, A, B].
    :return: Delta-E value.
    """
    delta_e = np.linalg.norm(lab1 - lab2)
    logging.info(f"Delta-E between the two average colors: {delta_e:.2f}")
    return delta_e

def lab_to_rgb(lab_image):
    """
    Convert LAB image to RGB PIL Image.

    :param lab_image: LAB color numpy array.
    :return: RGB PIL Image.
    """
    rgb_array = color.lab2rgb(lab_image)
    rgb_array = np.clip(rgb_array * 255, 0, 255).astype(np.uint8)
    return Image.fromarray(rgb_array)

def display_image(image, title='Image', figsize=(6,6)):
    """
    Display an image using matplotlib.

    :param image: PIL Image to display.
    :param title: Title of the image.
    :param figsize: Size of the figure.
    """
    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()

def color_difference_map(image1_lab, image2_lab, threshold=2):
    """
    Calculate and display the color difference (∆E) between two LAB images.

    :param image1_lab: Original LAB image.
    :param image2_lab: Faded LAB image.
    :param threshold: ∆E threshold for detectable fading.
    :return: ∆E difference map.
    """
    delta_e = np.linalg.norm(image1_lab - image2_lab, axis=2)
    detectable_fading = np.sum(delta_e > threshold)
    total_pixels = delta_e.size
    percentage_detectable = (detectable_fading / total_pixels) * 100
    logging.info(f"Percentage of image with detectable fading (∆E > {threshold}): {percentage_detectable:.2f}%")

    plt.figure(figsize=(8, 6))
    plt.imshow(delta_e, cmap='hot')
    plt.colorbar(label='∆E')
    plt.title("Pixel-by-Pixel Color Difference Map (∆E)")
    plt.axis('off')
    plt.show()

    return delta_e

def plot_histograms(image1, image2, title_suffix=''):
    """
    Plot RGB and LAB histograms for two images.

    :param image1: The first image (PIL Image format).
    :param image2: The second image (PIL Image format).
    :param title_suffix: Suffix to add to the titles.
    """
    image1_array = np.array(image1)
    image2_array = np.array(image2)

    # Flatten the RGB values for both images
    image1_flat = image1_array.reshape(-1, 3)
    image2_flat = image2_array.reshape(-1, 3)

    fig, axs = plt.subplots(2, 2, figsize=(15, 10))

    # RGB Histograms
    colors = ['Red', 'Green', 'Blue']
    channels = [0, 1, 2]
    colors_map = ['r', 'g', 'b']
    faded_colors_map = ['darkred', 'darkgreen', 'darkblue']

    for i, (color_name, channel, color_map) in enumerate(zip(colors, channels, colors_map)):
        axs[0, 0].hist(image1_flat[:, channel], bins=256, color=color_map, alpha=0.5, label=f'{color_name} (Original)')
        axs[0, 0].hist(image2_flat[:, channel], bins=256, color=faded_colors_map[i], alpha=0.5, label=f'{color_name} (Faded)')

    axs[0, 0].set_title(f'RGB Channel Histograms {title_suffix}')
    axs[0, 0].legend()

    # LAB Histograms for L*, A*, B*
    image1_lab = color.rgb2lab(image1_array.astype(np.float32) / 255.0).reshape(-1, 3)
    image2_lab = color.rgb2lab(image2_array.astype(np.float32) / 255.0).reshape(-1, 3)

    # L* Histogram
    axs[0, 1].hist(image1_lab[:, 0], bins=256, color='magenta', alpha=0.5, label='L* (Original)')
    axs[0, 1].hist(image2_lab[:, 0], bins=256, color='orange', alpha=0.5, label='L* (Faded)')
    axs[0, 1].set_title(f'L* (Lightness) Histogram {title_suffix}')
    axs[0, 1].legend()

    # A* Histogram
    axs[1, 0].hist(image1_lab[:, 1], bins=256, color='cyan', alpha=0.5, label='A* (Original)')
    axs[1, 0].hist(image2_lab[:, 1], bins=256, color='blue', alpha=0.5, label='A* (Faded)')
    axs[1, 0].set_title(f'A* Histogram {title_suffix}')
    axs[1, 0].legend()

    # B* Histogram
    axs[1, 1].hist(image1_lab[:, 2], bins=256, color='yellow', alpha=0.5, label='B* (Original)')
    axs[1, 1].hist(image2_lab[:, 2], bins=256, color='green', alpha=0.5, label='B* (Faded)')
    axs[1, 1].set_title(f'B* Histogram {title_suffix}')
    axs[1, 1].legend()

    plt.tight_layout()
    plt.show()

def display_average_color(image_lab, title='Average Color'):
    """
    Calculate and display the average color of a LAB image.

    :param image_lab: LAB color numpy array.
    :param title: Title for the color patch.
    :return: Average LAB values.
    """
    average_lab = image_lab.mean(axis=(0,1))
    average_rgb = color.lab2rgb(np.reshape(average_lab, (1,1,3))).reshape(1,1,3)
    average_rgb = np.clip(average_rgb, 0, 1)

    plt.figure(figsize=(2,2))
    plt.imshow(np.ones((100,100,3)) * average_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

    logging.info(f"{title}: L={average_lab[0]:.2f}, A={average_lab[1]:.2f}, B={average_lab[2]:.2f}")
    return average_lab

def find_closest_colors(dataset, lab_tree, query_lab, k=5):
    """
    Find the closest colors in the dataset to the query LAB color.

    :param dataset: pandas DataFrame containing LAB values and color names.
    :param lab_tree: KDTree built from LAB values.
    :param query_lab: List or array of [L, A, B] values.
    :param k: Number of closest colors to find.
    :return: pandas DataFrame of closest colors with distances.
    """
    distances, indices = lab_tree.query([query_lab], k=k)
    closest_colors = dataset.iloc[indices[0]]
    closest_colors = closest_colors.copy()
    closest_colors['Distance'] = distances[0]
    logging.info(f"Found {k} closest colors in the dataset.")
    return closest_colors

def display_closest_colors(closest_colors):
    """
    Display the closest colors as color patches with their names and distances.

    :param closest_colors: pandas DataFrame with color information and distances.
    """
    num_colors = len(closest_colors)
    fig, axs = plt.subplots(1, num_colors, figsize=(3*num_colors, 3))
    if num_colors == 1:
        axs = [axs]
    for i, (index, row) in enumerate(closest_colors.iterrows()):
        lab = np.array([row['L'], row['A'], row['B']]).reshape(1,1,3)
        rgb = color.lab2rgb(lab).reshape(3,)
        rgb = np.clip(rgb, 0, 1)
        color_patch = np.ones((100, 100, 3)) * rgb
        axs[i].imshow(color_patch)
        axs[i].axis('off')
        axs[i].set_title(f"{row['Color Name']}\nDist: {row['Distance']:.2f}")
    plt.show()

# -----------------------------
# Step 6: Main Execution Flow
# -----------------------------

def main():
    # Upload and load dataset
    csv_filename = upload_file("Please upload your LAB color dataset CSV file.", file_types=('.csv',))
    dataset = load_and_clean_dataset(csv_filename)

    # Create KDTree for LAB color matching
    lab_tree = KDTree(dataset[['L', 'A', 'B']].values)

    # Upload and process image
    original_image, original_lab = upload_and_process_image()

    # Display average color before fading
    average_lab_original = display_average_color(original_lab, title='Average Color Before Fading')

    # Train machine learning model
    synthetic_data = create_synthetic_data(num_samples=1000)
    model, scaler, mse = train_ml_model(synthetic_data)

    # Display model evaluation
    print(f"Mean Squared Error for Fading Prediction: {mse:.4f}")

    # Predict fading for sample lux hours (UV)
    sample_lux_hours = np.array([[50000]])  # Example lux hours
    predicted_fading_uv = predict_fading(model, scaler, sample_lux_hours)

    # Simulate UV exposure
    lab_uv_exposed = simulate_uv_exposure(original_lab, exposure_years=5)
    uv_exposed_image = lab_to_rgb(lab_uv_exposed)
    display_image(uv_exposed_image, title='Simulated UV Exposure (5 years)')

    # Predict fading for UV exposure
    fading_uv = predict_fading(model, scaler, np.array([[50000]]))

    # Apply fading to UV exposed image
    lab_uv_faded = apply_fading(lab_uv_exposed, fading_uv[0])
    uv_faded_image = lab_to_rgb(lab_uv_faded)
    display_image(uv_faded_image, title='Faded Image After UV Exposure Prediction')

    # Extract and compare average color after fading
    average_lab_faded = display_average_color(lab_uv_faded, title='Average Color After Fading')

    # Calculate Delta-E between original and faded average colors
    delta_e_average = compute_delta_e(average_lab_original, average_lab_faded)
    print(f"Delta-E between average colors: {delta_e_average:.2f}")

    # Find and display closest colors
    closest_colors = find_closest_colors(dataset, lab_tree, average_lab_faded, k=5)
    display_closest_colors(closest_colors)

    # Pixel-by-pixel color difference mapping
    color_diff_uv = color_difference_map(original_lab, lab_uv_faded, threshold=2)

    # Plot histograms for UV exposure simulation
    plot_histograms(original_image, uv_faded_image, title_suffix='(UV Exposure)')

    # Optional: Visible Light Exposure Simulation
    # Simulate visible light exposure
    lab_visible_exposed = simulate_visible_light_exposure(original_lab, exposure_years=5)
    visible_exposed_image = lab_to_rgb(lab_visible_exposed)
    display_image(visible_exposed_image, title='Simulated Visible Light Exposure (5 years)')

    # Predict fading for visible light exposure
    fading_visible = predict_fading(model, scaler, np.array([[50000]]))

    # Apply fading to visible light exposed image
    lab_visible_faded = apply_fading(lab_visible_exposed, fading_visible[0])
    visible_faded_image = lab_to_rgb(lab_visible_faded)
    display_image(visible_faded_image, title='Faded Image After Visible Light Exposure Prediction')

    # Plot histograms for visible light exposure simulation
    plot_histograms(visible_exposed_image, visible_faded_image, title_suffix='(Visible Light Exposure)')

    # Pixel-by-pixel color difference mapping for visible light
    color_diff_visible = color_difference_map(original_lab, lab_visible_faded, threshold=2)

if __name__ == "__main__":
    main()